In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from datetime import datetime, timedelta

In [2]:
import sys

# 获取当前 Python 的版本信息
python_version = sys.version
print(python_version)

3.11.4 (main, Jul  5 2023, 08:54:11) [Clang 14.0.6 ]


In [60]:
def merge_OT_SPC(curated_parm, curated_spc):
    
    # test
    # curated_parm = parm
    # curated_spc = spc

    # Step 1: 处理 spc 数据 筛选 EntryType 等于 3 的数据
    spc = curated_spc[curated_spc['EntryType'] == 3] # weight
    spc = spc[['DataTime', 'Item', 'Load', 'Actual']].copy()
    spc.rename(columns={'Actual': 'Weight'}, inplace=True)
    
    # 根据Item的首字母判断是否含有糖
    spc['Sugar'] = np.where(spc['Item'].str[0].isin(['D', 'W', 'R']), 'Sugar', 'Sugarfree')

    # OT 需要的列名列表
    required_columns = [
        "TS",
        "SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning",
        "CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches",
        "CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches",
        "CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap1stSizing.rActualPosition_inches",
        "CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapFinalSizing.rActualPosition_inches",
        "CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio",
        "CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint",
        "CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp",
        "CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp",
        "CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp"
    ]
    
    # Step 2: 处理 para 数据
    para = curated_parm.copy()

    # 将 Tag 为 "SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning" 的 Value 列中的 True 和 False 修改为 1 和 0
    para.loc[para['Tag'] == "SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning", 'Value'] = \
        para.loc[para['Tag'] == "SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning", 'Value'].apply(lambda x: 1 if x == 'True' else 0)

    # 筛选出 Tag 列中在 required_columns 中的元素
    para = para[para['Tag'].isin(required_columns[1:])]  # required_columns[1:] 是除去 'TS' 的其他列

    # 行转列
    para = para.pivot_table(index='TS', columns='Tag', values='Value', aggfunc='mean').reset_index()

    # 确保所有必需的列都存在，缺失的列用 NA 填充
    for col in required_columns:
        if col not in para.columns:
            para[col] = np.nan

    # 选择列并按顺序排列
    para = para[required_columns]
    para.fillna(method='ffill', inplace=True)

    # 提取最新一条记录
    latest_dict = para.iloc[-1].to_dict()

    para['TS'] = pd.to_datetime(para['TS']) + timedelta(minutes=1)

    # 将 TS 列重命名为 Date
    para.rename(columns={'TS': 'DataTime'}, inplace=True)

    # Step 3: 进行数据合并与计算
    # 当在 spc 数据框中找到一个 Date 值时，它会在 para 数据框中查找等于或早于该 Date 值的最近一行进行匹配。
    # 因此，如果 spc 中某个 Date 的值在 para 中找不到完全相同的 Date，则会回退到最接近但早于它的那个 Date 进行匹配。
    merge = pd.merge_asof(spc.sort_values('DataTime'), 
                          para.sort_values('DataTime'), 
                          on='DataTime', 
                          direction='backward')

    merge['Prev_Weight'] = merge['Weight'].shift(1)


    # 计算过去 5, 15 和 30 分钟的均值，不包含当前重量
    # closed='left'：指定窗口左闭右开，这意味着在计算滚动平均值时，窗口会排除当前记录的值，只考虑当前记录之前的值。
    # df[col] 是一个 Series 对象，而不是一个 DataFrame，因此它无法识别 on='DataTime' 选项。
    # 在 DataFrame 上调用 rolling，而不是在 Series 上调用。
    def calculate_avg_weight(df, minutes_back, col):
        if df[col].isna().all():
            return df[col]  # 如果全是空值，返回原列
        # 使用 DataFrame 而不是 Series 进行 rolling 操作
        return df.rolling(f'{minutes_back}T', on='DataTime', closed='left', min_periods=1)[col].mean()

    # 示例：对每个列进行检查并计算滚动平均值
    merge['Avg_Weight_5min'] = calculate_avg_weight(merge, 5, 'Weight')
    merge['Avg_Weight_15min'] = calculate_avg_weight(merge, 15, 'Weight')
    merge['Avg_Weight_30min'] = calculate_avg_weight(merge, 30, 'Weight')

    # 仅处理 key_columns 中除了前两个元素以外的所有列
    for col in required_columns[2:]:
        merge[f'Prev_{col}'] = merge[col].shift(1)

    # 按照指定顺序重新排列列
    # merge 数据框中的列会按照以下顺序排列：
    # 固定顺序的列（Date, Load, Item, Sugar, Weight, Prev_Weight, Avg_Weight_5min, Avg_Weight_15min, Avg_Weight_30min）。
    # 动态生成的列，这些列以原列名和对应的 Prev_ 前缀列名成对排列。

    # merge = merge[['DataTime', 'Load', 'Item', 'Sugar', 
    #               'Weight', 'Prev_Weight', 'Avg_Weight_5min', 'Avg_Weight_15min', 'Avg_Weight_30min'] + 
     #             [col for pair in zip(key_columns, [f'Prev_{col}' for col in key_columns]) for col in pair]]

    # 构建固定的列列表
    base_columns = ['DataTime', 'Load', 'Item', 'Sugar', 
                    'Weight', 'Prev_Weight', 'Avg_Weight_5min', 'Avg_Weight_15min', 'Avg_Weight_30min']

    # 动态生成要选择的列，确保选择的列存在于 merge 中
    dynamic_columns = [col for pair in zip(required_columns, [f'Prev_{col}' for col in required_columns]) 
                       for col in pair if col in merge.columns]

    # 合并固定列和动态生成的列
    selected_columns = base_columns + dynamic_columns

    # 选择存在的列，不会因缺少某些列而报错
    merge = merge[selected_columns]

    # 返回结果
    return merge, latest_dict

In [61]:
def process_etl_data(now=None,
                     parm_file="curated_parm.csv",
                     spc_file="curated_spc.csv",
                     prev_merge_file="prev_merge.csv",
                     merge_OT_SPC=None,
                     output_file="merge_final_py.csv"):
    
    # Step 1: 读取当前ETL生成的15秒的数据 + 上一个合并文件
    parm = pd.read_csv(parm_file)
    spc = pd.read_csv(spc_file)
    prev_merge = pd.read_csv(prev_merge_file)
    
    # 转化时间格式为没有时区信息的 pandas.Timestamp 对象
    parm['TS'] = pd.to_datetime(parm['TS'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)
    spc['DataTime'] = pd.to_datetime(spc['DataTime'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)
    prev_merge['DataTime'] = pd.to_datetime(prev_merge['DataTime'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)
    
    # 删除 parm 中 TS 列为空值的行
    parm = parm.dropna(subset=['TS'])

    # Step 2: 直接进行合并，获取 merge 和 merge_latest_dict
    merge_new, merge_latest_dict = merge_OT_SPC(parm, spc)
    
    # Step 3: 筛选数据，去掉既不是最近30分钟又不是最近10次的数据
    recent_30min = now - timedelta(minutes=30)

    # 保留最近30分钟的数据
    merge_recent_30min = merge_new[merge_new['DataTime'] >= recent_30min]
    
    # 保留最近10次的记录
    merge_recent_10 = merge_new.tail(10)
    
    # 选择两者中的较大集合：去掉既不是最近30分钟又不是最近10次的数据作为当前15秒的merge结果
    if len(merge_recent_30min) >= len(merge_recent_10):
        merge_final = merge_recent_30min
    else:
        merge_final = merge_recent_10
    
#####################################################################################

    # Step 4: 将 merge_final 的第一行的 DateTime 用 prev_merge 中的对应行替换
    first_datetime = merge_final.iloc[0]['DataTime']
    prev_row = prev_merge[prev_merge['DataTime'] == first_datetime]
    if not prev_row.empty:
        merge_final.iloc[0] = prev_row.iloc[0]
    
    # Step 5: 对除了前9列的所有列中的NA数值进行填充
    merge_final.iloc[:, 9:] = merge_final.iloc[:, 9:].fillna(method='ffill')

#####################################################################################

    # 保存 merge_final 到 output_file
    merge_final.to_csv(output_file, index=False)
    
    # Step 5: 获取最新一条数据（离当前时间最近的一条）
    latest_data = merge_final.sort_values(by='DataTime', ascending=False).iloc[0]
    
    # 生成 dictionary {列名：当前值}
    latest_dict = latest_data.to_dict()
    
    # 输出两个latest_dict进行对比
    print("Merge Latest Dict:")
    print(merge_latest_dict)
    
    print("\nProcess ETL Data Latest Dict:")
    print(latest_dict)
    
    # 返回 merge_latest_dict 和 latest_dict
    return merge_latest_dict, latest_dict

# 运行示例
merge_latest_dict, latest_dict = process_etl_data(now=pd.to_datetime("2024-08-26 15:00:00", format="%Y-%m-%d %H:%M:%S").tz_localize(None),
                                                  parm_file="curated_parm.csv",
                                                  spc_file="curated_spc.csv",
                                                  prev_merge_file="merge.csv",
                                                  merge_OT_SPC=merge_OT_SPC,
                                                  output_file="merge_final_py.csv")


print("Returned Merge Latest Dict:")
print(merge_latest_dict)

print("\nReturned Process ETL Data Latest Dict:")
print(latest_dict)

Merge Latest Dict:
{'TS': Timestamp('2024-08-26 15:04:16'), 'SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning': 0.0, 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches': 0.069, 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches': 0.075, 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap1stSizing.rActualPosition_inches': 0.109, 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapFinalSizing.rActualPosition_inches': 0.065, 'CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio': 182.7, 'CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint': -20.0, 'CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp': -20.3, 'CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp': -20.9, 'CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp': 50.6}

Process ETL Data Latest Dict:
{'DataTime': Timestamp('2024-08-26 15:04:00'), 'Load': 105.0, 'Item': 'EXCW（益达西瓜-NCS）', 'Sugar': 'Sug

In [ ]:
# # original
# def process_etl_data(now=None,
#                      parm_file="curated_parm.csv",
#                      spc_file="curated_spc.csv",
#                      prev_merge_file="merge.csv",
#                      merge_OT_SPC=None,
#                      output_file="merge_final_py2.csv"):
    
#     # Step 1: 读取当前ETL生成的15秒的数据 + 上一个合并文件
#     parm = pd.read_csv(parm_file)
#     spc = pd.read_csv(spc_file)
#     prev_merge = pd.read_csv(prev_merge_file)
    
#     # 转化时间格式为没有时区信息的 pandas.Timestamp 对象
#     parm['TS'] = pd.to_datetime(parm['TS'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)
#     spc['DataTime'] = pd.to_datetime(spc['DataTime'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)
#     prev_merge['DataTime'] = pd.to_datetime(prev_merge['DataTime'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)
    
#     # 删除 parm 中 TS 列为空值的行
#     parm = parm.dropna(subset=['TS'])

#     # Step 2: 直接进行合并，获取 merge 和 latest_dict
#     merge_new, merge_latest_dict = merge_OT_SPC(parm, spc)
    
#     # Step 3: 筛选数据，去掉既不是最近30分钟又不是最近10次的数据
#     recent_30min = now - timedelta(minutes=30)

#     # 保留最近30分钟的数据
#     merge_recent_30min = merge_new[merge_new['DataTime'] >= recent_30min]
    
#     # 保留最近10次的记录
#     merge_recent_10 = merge_new.tail(10)
    
#     # 选择两者中的较大集合：去掉既不是最近30分钟又不是最近10次的数据作为当前15秒的merge结果
#     if len(merge_recent_30min) >= len(merge_recent_10):
#         merge_final = merge_recent_30min
#     else:
#         merge_final = merge_recent_10
    
#     # 保存 merge_final 到 output_file
#     merge_final.to_csv(output_file, index=False)
    
#     # Step 5: 获取最新一条数据（离当前时间最近的一条）
#     latest_data = merge_final.sort_values(by='DataTime', ascending=False).iloc[0]
    
#     # 生成 dictionary {列名：当前值}
#     latest_dict = latest_data.to_dict()
    
#     # 输出两个latest_dict进行对比
#     print("Merge Latest Dict:")
#     print(merge_latest_dict)
    
#     print("\nProcess ETL Data Latest Dict:")
#     print(latest_dict)
    
#     # 返回最新数据的 dictionary
#     return latest_dict

# # 运行示例
# result = process_etl_data(now=pd.to_datetime("2024-07-06 06:20:00", format="%Y-%m-%d %H:%M:%S").tz_localize(None),
#                           parm_file="curated_parm.csv",
#                           spc_file="curated_spc.csv",
#                           prev_merge_file="merge.csv",
#                           merge_OT_SPC=merge_OT_SPC,
#                           output_file="merge_final_py2.csv")

# print(result)

In [166]:
parm_file="curated_parm.csv"
spc_file="curated_spc.csv"
# prev_merge_file="merge.csv"

# Step 1: 读取当前ETL生成的15秒的数据 + 上一个合并文件
parm = pd.read_csv(parm_file)
spc = pd.read_csv(spc_file)
# prev_merge = pd.read_csv(prev_merge_file)

# 转化时间格式为没有时区信息的 pandas.Timestamp 对象
parm['TS'] = pd.to_datetime(parm['TS'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)
spc['DataTime'] = pd.to_datetime(spc['DataTime'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)
# prev_merge['DataTime'] = pd.to_datetime(prev_merge['DataTime'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)

# 删除 parm 中 TS 列为空值的行
parm = parm.dropna(subset=['TS'])

curated_parm = parm
curated_spc = spc

# Step 1: 处理 spc 数据
spc = curated_spc[['DataTime', 'Item', 'Load', 'Actual']].copy()
spc.rename(columns={'Actual': 'Weight'}, inplace=True)

# 根据Item的首字母判断是否含有糖
spc['Sugar'] = np.where(spc['Item'].str[0].isin(['D', 'W', 'R']), 'Sugar', 'Sugarfree')

# 需要的列名列表
required_columns = [
    "TS",
    "SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning",
    "CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches",
    "CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches",
    "CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap1stSizing.rActualPosition_inches",
    "CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapFinalSizing.rActualPosition_inches",
    "CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio",
    "CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint",
    "CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp",
    "CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp",
    "CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp"
]

# Step 2: 处理 para 数据
para = curated_parm.copy()
#   para.rename(columns={'TS': 'Date'}, inplace=True)


In [167]:
para.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1885 entries, 0 to 1884
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   IOTDeviceID  0 non-null      float64       
 1   SiteId       1885 non-null   object        
 2   LineId       1885 non-null   object        
 3   SensorId     1885 non-null   object        
 4   MachineId    0 non-null      float64       
 5   Tag          1885 non-null   object        
 6   Value        1885 non-null   float64       
 7   TS           1885 non-null   datetime64[ns]
 8   uuid         1885 non-null   object        
 9   TS2          1885 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 147.4+ KB


In [168]:
para.head()

,IOTDeviceID,SiteId,LineId,SensorId,MachineId,Tag,Value,TS,uuid,TS2
0,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-15.7,2024-08-21 09:55:00,9bda31a6-5b57-4c24-b659-425c8245537b,2024-08-21 09:55:00.546 +08:00
1,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-15.7,2024-08-21 09:55:15,b1bc5d82-32a2-4ca7-870f-307e2cf56f30,2024-08-21 09:55:15.409 +08:00
2,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-15.7,2024-08-21 09:55:30,7c73c4d3-7471-44c2-af8c-84bc8d0afe30,2024-08-21 09:55:30.485 +08:00
3,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-15.8,2024-08-21 09:55:45,c5fbedd5-f187-4b3a-81e5-35c881707436,2024-08-21 09:55:45.403 +08:00
4,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-15.8,2024-08-21 09:56:00,b5e5837e-4823-456e-aadc-143c1618d22d,2024-08-21 09:56:00.555 +08:00


In [170]:
para = para.pivot_table(index='TS', columns='Tag', values='Value', aggfunc='mean').reset_index()

In [172]:
para.head()

Tag,TS,CG STI.CG STI.LoafGum.LoafGum01MaxTemp,CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1OutletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2OutletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rGumEntranceTemperature,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rGumExitTempLeft,...,CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches,CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches,CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapBullRoll.rActualPosition_inches,CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapFinalSizing.rActualPosition_inches,SFBMix.PLC_BOSCH EXTRUDER.DB_Data_Exchange.EXT_LB_Temp_RealValue,SFBMix.PLC_BOSCH EXTRUDER.DB_Data_Exchange.EXT_LB_Temp_SP,SFBMix.PLC_BOSCH EXTRUDER.DB_Data_Exchange.EXT_PH_Temp_RealValue,SFBMix.PLC_BOSCH EXTRUDER.DB_Data_Exchange.EXT_PH_Temp_SP,SFBMix.PLC_BOSCH EXTRUDER.DB_Data_Exchange.EXT_UB_Temp_RealValue,SFBMix.PLC_BOSCH EXTRUDER.DB_Data_Exchange.EXT_UB_Temp_SP
0,2024-08-21 09:33:14,NaN,NaN,-15.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.067,0.064,0.188,0.063,NaN,40.0,NaN,52.0,NaN,40.0
1,2024-08-21 09:33:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-08-21 09:40:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-08-21 09:40:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-08-21 09:40:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
para.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138 entries, 0 to 1137
Data columns (total 11 columns):
 #   Column                                                                            Non-Null Count  Dtype         
---  ------                                                                            --------------  -----         
 0   DataTime                                                                          1138 non-null   datetime64[ns]
 1   SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning          0 non-null      float64       
 2   CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches    1138 non-null   float64       
 3   CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches    1138 non-null   float64       
 4   CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap1stSizing.rActualPosition_inches    1138 non-null   float64       
 5   CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapFinalSizing.rAct

In [180]:
# 假设时间列名为 'TS'
timestamp_to_print = pd.to_datetime("2024-08-21 10:01:53")

# 使用布尔索引筛选出指定时间的数据
selected_data = para[para['DataTime'] == timestamp_to_print]

# 打印筛选出的数据
print(selected_data)

Tag            DataTime  \
796 2024-08-21 10:01:53   

Tag  SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning  \
796                                                NaN                          

Tag  CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches  \
796                                              0.064                                

Tag  CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches  \
796                                              0.067                                

Tag  CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap1stSizing.rActualPosition_inches  \
796                                              0.098                                

Tag  CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapFinalSizing.rActualPosition_inches  \
796                                              0.063                                  

Tag  CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio  \
7

In [181]:
latest_dict = para.iloc[-1].to_dict()
print(latest_dict)

{'DataTime': Timestamp('2024-08-21 10:11:13'), 'SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning': nan, 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches': 0.064, 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches': 0.067, 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap1stSizing.rActualPosition_inches': 0.098, 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapFinalSizing.rActualPosition_inches': 0.063, 'CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio': 211.1, 'CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint': -15.0, 'CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp': -15.7, 'CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp': -16.3, 'CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp': 48.2}


In [31]:
parm_file="curated_parm.csv"
spc_file="curated_spc.csv"

# prev_merge_file="merge.csv"

# Step 1: 读取当前ETL生成的15秒的数据 + 上一个合并文件
parm = pd.read_csv(parm_file)
spc = pd.read_csv(spc_file)
# prev_merge = pd.read_csv(prev_merge_file)

# 转化时间格式为没有时区信息的 pandas.Timestamp 对象
parm['TS'] = pd.to_datetime(parm['TS'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)
spc['DataTime'] = pd.to_datetime(spc['DataTime'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)
# prev_merge['DataTime'] = pd.to_datetime(prev_merge['DataTime'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)

# 删除 parm 中 TS 列为空值的行
parm = parm.dropna(subset=['TS'])

curated_parm = parm
curated_spc = spc

# Step 1: 筛选 EntryType 等于 3 的数据
spc = curated_spc[curated_spc['EntryType'] == 3] # weight

# Step 2: 处理 spc 数据
spc = spc[['DataTime', 'Item', 'Load', 'Actual']].copy()
spc.rename(columns={'Actual': 'Weight'}, inplace=True)

# 根据Item的首字母判断是否含有糖
spc['Sugar'] = np.where(spc['Item'].str[0].isin(['D', 'W', 'R']), 'Sugar', 'Sugarfree')

# 需要的列名列表
required_columns = [
    "TS",
    "SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning",
    "CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches",
    "CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches",
    "CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap1stSizing.rActualPosition_inches",
    "CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapFinalSizing.rActualPosition_inches",
    "CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio",
    "CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint",
    "CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp",
    "CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp",
    "CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp"
]

In [34]:
# Step 2: 处理 para 数据
para = curated_parm.copy()

In [35]:
# 将 Tag 为 "SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning" 的 Value 列中的 True 和 False 修改为 1 和 0
para.loc[para['Tag'] == "SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning", 'Value'] = \
    para.loc[para['Tag'] == "SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning", 'Value'].apply(lambda x: 1 if x == 'True' else 0)


In [36]:
para.head()

,IOTDeviceID,SiteId,LineId,SensorId,MachineId,Tag,Value,TS,uuid,TS2
0,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-20.2,2024-08-26 14:49:15,51aff997-9a93-4aea-a21c-9d95d55ed115,2024-08-26 14:49:15.692 +08:00
1,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-20.2,2024-08-26 14:49:30,6a1c61c4-f34a-493a-8204-35add74e41c0,2024-08-26 14:49:30.430 +08:00
2,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-20.2,2024-08-26 14:49:45,97947e52-e5e0-4386-9c6f-8257c7933360,2024-08-26 14:49:51.548 +08:00
3,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-20.2,2024-08-26 14:50:00,a0e751c5-bf56-4ec8-8f58-87074bf6f743,2024-08-26 14:50:00.579 +08:00
4,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-20.2,2024-08-26 14:50:15,6ab6b757-18a5-46b6-87bc-bf87a7e4c828,2024-08-26 14:50:15.489 +08:00


In [37]:
# 筛选出 Tag 列中在 required_columns 中的元素
para = para[para['Tag'].isin(required_columns[1:])]  # required_columns[1:] 是除去 'TS' 的其他列

In [38]:
para.head()

,IOTDeviceID,SiteId,LineId,SensorId,MachineId,Tag,Value,TS,uuid,TS2
0,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-20.2,2024-08-26 14:49:15,51aff997-9a93-4aea-a21c-9d95d55ed115,2024-08-26 14:49:15.692 +08:00
1,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-20.2,2024-08-26 14:49:30,6a1c61c4-f34a-493a-8204-35add74e41c0,2024-08-26 14:49:30.430 +08:00
2,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-20.2,2024-08-26 14:49:45,97947e52-e5e0-4386-9c6f-8257c7933360,2024-08-26 14:49:51.548 +08:00
3,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-20.2,2024-08-26 14:50:00,a0e751c5-bf56-4ec8-8f58-87074bf6f743,2024-08-26 14:50:00.579 +08:00
4,NaN,YNG,<not set>,opcua,NaN,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variable...,-20.2,2024-08-26 14:50:15,6ab6b757-18a5-46b6-87bc-bf87a7e4c828,2024-08-26 14:50:15.489 +08:00


In [39]:
# 行转列
# para_pivoted = filtered_para.pivot(index='TS', columns='Tag', values='Value').reset_index()

para = para.pivot_table(index='TS', columns='Tag', values='Value', aggfunc='mean').reset_index()

In [40]:
para.head()

Tag,TS,CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap1stSizing.rActualPosition_inches,CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches,CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches,CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapFinalSizing.rActualPosition_inches,SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning
0,2024-08-26 14:27:08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2024-08-26 14:34:15,NaN,NaN,NaN,NaN,182.7,NaN,0.075,0.069,0.065,NaN
2,2024-08-26 14:34:16,NaN,NaN,NaN,NaN,NaN,0.109,NaN,NaN,NaN,NaN
3,2024-08-26 14:34:21,NaN,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-08-26 14:34:29,NaN,-20.0,NaN,NaN,NaN,0.109,0.075,0.069,NaN,NaN


In [41]:
# 确保所有必需的列都存在，缺失的列用 NA 填充
for col in required_columns:
    if col not in para.columns:
        para[col] = np.nan

# 选择列并按顺序排列
para = para[required_columns]
para.fillna(method='ffill', inplace=True)

In [42]:
para.head()

Tag,TS,SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning,CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches,CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches,CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap1stSizing.rActualPosition_inches,CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapFinalSizing.rActualPosition_inches,CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp
0,2024-08-26 14:27:08,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-26 14:34:15,0.0,0.069,0.075,NaN,0.065,182.7,NaN,NaN,NaN,NaN
2,2024-08-26 14:34:16,0.0,0.069,0.075,0.109,0.065,182.7,NaN,NaN,NaN,NaN
3,2024-08-26 14:34:21,0.0,0.069,0.075,0.109,0.065,182.7,-20.0,NaN,NaN,NaN
4,2024-08-26 14:34:29,0.0,0.069,0.075,0.109,0.065,182.7,-20.0,NaN,NaN,NaN


In [43]:
# 提取最新一条记录
latest_dict = para.iloc[-1].to_dict()

latest_dict

{'TS': Timestamp('2024-08-26 15:04:16'),
 'SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning': 0.0,
 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches': 0.069,
 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches': 0.075,
 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap1stSizing.rActualPosition_inches': 0.109,
 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapFinalSizing.rActualPosition_inches': 0.065,
 'CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio': 182.7,
 'CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint': -20.0,
 'CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp': -20.3,
 'CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp': -20.9,
 'CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp': 50.6}

In [44]:
# later delet this: just to fill the NA
#  para.fillna(method='bfill', inplace=True)
para['TS'] = pd.to_datetime(para['TS']) + timedelta(minutes=1)

# 将 TS 列重命名为 Date
para.rename(columns={'TS': 'DataTime'}, inplace=True)


In [45]:
# Step 3: 进行数据合并与计算
# 当在 spc 数据框中找到一个 Date 值时，它会在 para 数据框中查找等于或早于该 Date 值的最近一行进行匹配。
# 因此，如果 spc 中某个 Date 的值在 para 中找不到完全相同的 Date，则会回退到最接近但早于它的那个 Date 进行匹配。
merge = pd.merge_asof(spc.sort_values('DataTime'), 
                      para.sort_values('DataTime'), 
                      on='DataTime', 
                      direction='backward')

merge['Prev_Weight'] = merge['Weight'].shift(1)


# 计算过去 5, 15 和 30 分钟的均值，不包含当前重量
# closed='left'：指定窗口左闭右开，这意味着在计算滚动平均值时，窗口会排除当前记录的值，只考虑当前记录之前的值。
# df[col] 是一个 Series 对象，而不是一个 DataFrame，因此它无法识别 on='DataTime' 选项。
# 在 DataFrame 上调用 rolling，而不是在 Series 上调用。
def calculate_avg_weight(df, minutes_back, col):
    if df[col].isna().all():
        return df[col]  # 如果全是空值，返回原列
    # 使用 DataFrame 而不是 Series 进行 rolling 操作
    return df.rolling(f'{minutes_back}T', on='DataTime', closed='left', min_periods=1)[col].mean()

# 示例：对每个列进行检查并计算滚动平均值
merge['Avg_Weight_5min'] = calculate_avg_weight(merge, 5, 'Weight')
merge['Avg_Weight_15min'] = calculate_avg_weight(merge, 15, 'Weight')
merge['Avg_Weight_30min'] = calculate_avg_weight(merge, 30, 'Weight')

# 仅处理 key_columns 中除了前两个元素以外的所有列
for col in required_columns[2:]:
    merge[f'Prev_{col}'] = merge[col].shift(1)

# 按照指定顺序重新排列列
# merge 数据框中的列会按照以下顺序排列：
# 固定顺序的列（Date, Load, Item, Sugar, Weight, Prev_Weight, Avg_Weight_5min, Avg_Weight_15min, Avg_Weight_30min）。
# 动态生成的列，这些列以原列名和对应的 Prev_ 前缀列名成对排列。

# merge = merge[['DataTime', 'Load', 'Item', 'Sugar', 
#               'Weight', 'Prev_Weight', 'Avg_Weight_5min', 'Avg_Weight_15min', 'Avg_Weight_30min'] + 
 #             [col for pair in zip(key_columns, [f'Prev_{col}' for col in key_columns]) for col in pair]]

# 构建固定的列列表
base_columns = ['DataTime', 'Load', 'Item', 'Sugar', 
                'Weight', 'Prev_Weight', 'Avg_Weight_5min', 'Avg_Weight_15min', 'Avg_Weight_30min']

# 动态生成要选择的列，确保选择的列存在于 merge 中
dynamic_columns = [col for pair in zip(required_columns, [f'Prev_{col}' for col in required_columns]) 
                   for col in pair if col in merge.columns]

# 合并固定列和动态生成的列
selected_columns = base_columns + dynamic_columns

# 选择存在的列，不会因缺少某些列而报错
merge = merge[selected_columns]

In [47]:
merge

,DataTime,Load,Item,Sugar,Weight,Prev_Weight,Avg_Weight_5min,Avg_Weight_15min,Avg_Weight_30min,SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning,...,CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp,Prev_CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp
0,2024-08-26 14:34:26,90.0,EXCW（益达西瓜-NCS）,Sugarfree,35.20,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-26 14:34:28,90.0,EXCW（益达西瓜-NCS）,Sugarfree,35.13,35.20,35.20,35.200000,35.200000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-08-26 14:42:37,95.0,EXCW（益达西瓜-NCS）,Sugarfree,35.03,35.13,NaN,35.165000,35.165000,0.0,...,182.7,NaN,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-08-26 14:52:52,100.0,EXCW（益达西瓜-NCS）,Sugarfree,35.37,35.03,NaN,35.030000,35.120000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,NaN,-20.9,NaN,51.0,NaN
4,2024-08-26 14:55:36,100.0,EXCW（益达西瓜-NCS）,Sugarfree,35.30,35.37,35.37,35.200000,35.182500,0.0,...,182.7,182.7,-20.0,-20.0,-20.2,-20.3,-20.7,-20.9,51.2,51.0
5,2024-08-26 15:02:20,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.22,35.30,NaN,35.335000,35.206000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.2,-20.8,-20.7,50.9,51.2
6,2024-08-26 15:02:25,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.22,35.22,35.22,35.296667,35.208333,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.3,-20.8,-20.8,50.9,50.9
7,2024-08-26 15:04:00,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.20,35.22,35.22,35.277500,35.210000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.3,-20.8,-20.8,50.9,50.9


In [48]:
merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 28 columns):
 #   Column                                                                                 Non-Null Count  Dtype         
---  ------                                                                                 --------------  -----         
 0   DataTime                                                                               8 non-null      datetime64[ns]
 1   Load                                                                                   8 non-null      float64       
 2   Item                                                                                   8 non-null      object        
 3   Sugar                                                                                  8 non-null      object        
 4   Weight                                                                                 8 non-null      float64       
 5   Prev_Weight                      

In [49]:
merge.to_csv('merge.csv', index=False)

In [50]:
prev_merge_file="prev_merge.csv"

prev_merge = pd.read_csv(prev_merge_file)
    
prev_merge['DataTime'] = pd.to_datetime(prev_merge['DataTime'], format="%Y-%m-%d %H:%M:%S", utc=True).dt.tz_localize(None)

In [51]:
prev_merge

,DataTime,Load,Item,Sugar,Weight,Prev_Weight,Avg_Weight_5min,Avg_Weight_15min,Avg_Weight_30min,SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning,...,CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp,Prev_CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp
0,2024-08-26 14:34:26,90.0,EXCW（益达西瓜-NCS）,Sugarfree,35.20,NaN,NaN,NaN,NaN,0.0,...,170.0,170.0,-15.0,-15.0,-17.0,-25.0,-25.0,-21.0,40.0,50.0
1,2024-08-26 14:34:28,90.0,EXCW（益达西瓜-NCS）,Sugarfree,35.13,35.20,35.20,35.200000,35.200000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-08-26 14:42:37,95.0,EXCW（益达西瓜-NCS）,Sugarfree,35.03,35.13,NaN,35.165000,35.165000,0.0,...,182.7,NaN,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-08-26 14:52:52,100.0,EXCW（益达西瓜-NCS）,Sugarfree,35.37,35.03,NaN,35.030000,35.120000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,NaN,-20.9,NaN,51.0,NaN
4,2024-08-26 14:55:36,100.0,EXCW（益达西瓜-NCS）,Sugarfree,35.30,35.37,35.37,35.200000,35.182500,0.0,...,182.7,182.7,-20.0,-20.0,-20.2,-20.3,-20.7,-20.9,51.2,51.0
5,2024-08-26 15:02:20,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.22,35.30,NaN,35.335000,35.206000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.2,-20.8,-20.7,50.9,51.2
6,2024-08-26 15:02:25,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.22,35.22,35.22,35.296667,35.208333,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.3,-20.8,-20.8,50.9,50.9
7,2024-08-26 15:04:00,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.20,35.22,35.22,35.277500,35.210000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.3,-20.8,-20.8,50.9,50.9


In [53]:
# Step 3: 筛选数据，去掉既不是最近30分钟又不是最近10次的数据
now = pd.to_datetime("2024-08-26 15:00:00", format="%Y-%m-%d %H:%M:%S").tz_localize(None)

recent_30min = now - timedelta(minutes=30)

# 保留最近30分钟的数据
merge_recent_30min = merge[merge['DataTime'] >= recent_30min]

# 保留最近10次的记录
merge_recent_10 = merge.tail(10)

# 选择两者中的较大集合：去掉既不是最近30分钟又不是最近10次的数据作为当前15秒的merge结果
if len(merge_recent_30min) >= len(merge_recent_10):
    merge_final = merge_recent_30min
else:
    merge_final = merge_recent_10

In [54]:
merge_final

,DataTime,Load,Item,Sugar,Weight,Prev_Weight,Avg_Weight_5min,Avg_Weight_15min,Avg_Weight_30min,SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning,...,CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp,Prev_CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp
0,2024-08-26 14:34:26,90.0,EXCW（益达西瓜-NCS）,Sugarfree,35.20,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-08-26 14:34:28,90.0,EXCW（益达西瓜-NCS）,Sugarfree,35.13,35.20,35.20,35.200000,35.200000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-08-26 14:42:37,95.0,EXCW（益达西瓜-NCS）,Sugarfree,35.03,35.13,NaN,35.165000,35.165000,0.0,...,182.7,NaN,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-08-26 14:52:52,100.0,EXCW（益达西瓜-NCS）,Sugarfree,35.37,35.03,NaN,35.030000,35.120000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,NaN,-20.9,NaN,51.0,NaN
4,2024-08-26 14:55:36,100.0,EXCW（益达西瓜-NCS）,Sugarfree,35.30,35.37,35.37,35.200000,35.182500,0.0,...,182.7,182.7,-20.0,-20.0,-20.2,-20.3,-20.7,-20.9,51.2,51.0
5,2024-08-26 15:02:20,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.22,35.30,NaN,35.335000,35.206000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.2,-20.8,-20.7,50.9,51.2
6,2024-08-26 15:02:25,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.22,35.22,35.22,35.296667,35.208333,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.3,-20.8,-20.8,50.9,50.9
7,2024-08-26 15:04:00,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.20,35.22,35.22,35.277500,35.210000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.3,-20.8,-20.8,50.9,50.9


In [55]:
# Step 4: 将 merge_final 的第一行的 DateTime 用 prev_merge 中的对应行替换
first_datetime = merge_final.iloc[0]['DataTime']
first_datetime

Timestamp('2024-08-26 14:34:26')

In [56]:
prev_row = prev_merge[prev_merge['DataTime'] == first_datetime]

prev_row

,DataTime,Load,Item,Sugar,Weight,Prev_Weight,Avg_Weight_5min,Avg_Weight_15min,Avg_Weight_30min,SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning,...,CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp,Prev_CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp
0,2024-08-26 14:34:26,90.0,EXCW（益达西瓜-NCS）,Sugarfree,35.2,NaN,NaN,NaN,NaN,0.0,...,170.0,170.0,-15.0,-15.0,-17.0,-25.0,-25.0,-21.0,40.0,50.0


In [57]:
if not prev_row.empty:
    merge_final.iloc[0] = prev_row.iloc[0]

merge_final 

,DataTime,Load,Item,Sugar,Weight,Prev_Weight,Avg_Weight_5min,Avg_Weight_15min,Avg_Weight_30min,SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning,...,CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp,Prev_CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp
0,2024-08-26 14:34:26,90.0,EXCW（益达西瓜-NCS）,Sugarfree,35.20,NaN,NaN,NaN,NaN,0.0,...,170.0,170.0,-15.0,-15.0,-17.0,-25.0,-25.0,-21.0,40.0,50.0
1,2024-08-26 14:34:28,90.0,EXCW（益达西瓜-NCS）,Sugarfree,35.13,35.20,35.20,35.200000,35.200000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-08-26 14:42:37,95.0,EXCW（益达西瓜-NCS）,Sugarfree,35.03,35.13,NaN,35.165000,35.165000,0.0,...,182.7,NaN,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-08-26 14:52:52,100.0,EXCW（益达西瓜-NCS）,Sugarfree,35.37,35.03,NaN,35.030000,35.120000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,NaN,-20.9,NaN,51.0,NaN
4,2024-08-26 14:55:36,100.0,EXCW（益达西瓜-NCS）,Sugarfree,35.30,35.37,35.37,35.200000,35.182500,0.0,...,182.7,182.7,-20.0,-20.0,-20.2,-20.3,-20.7,-20.9,51.2,51.0
5,2024-08-26 15:02:20,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.22,35.30,NaN,35.335000,35.206000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.2,-20.8,-20.7,50.9,51.2
6,2024-08-26 15:02:25,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.22,35.22,35.22,35.296667,35.208333,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.3,-20.8,-20.8,50.9,50.9
7,2024-08-26 15:04:00,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.20,35.22,35.22,35.277500,35.210000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.3,-20.8,-20.8,50.9,50.9


In [58]:
# Step 5: 对除了前9列的所有列中的NA数值进行填充
merge_final.iloc[:, 9:] = merge_final.iloc[:, 9:].fillna(method='ffill')

merge_final

,DataTime,Load,Item,Sugar,Weight,Prev_Weight,Avg_Weight_5min,Avg_Weight_15min,Avg_Weight_30min,SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning,...,CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Scoring.SRV_CrossScore.rSetpoint_Ratio,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rChillerSetpoint,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum1InletTemp,CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,Prev_CG_Sheeting.CG_Sheeting.dbHMI.Cooling.Variables.rDrum2InletTemp,CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp,Prev_CG_Sheeting.CG_Sheeting.Variables.rGumExtruderExitGumTemp
0,2024-08-26 14:34:26,90.0,EXCW（益达西瓜-NCS）,Sugarfree,35.20,NaN,NaN,NaN,NaN,0.0,...,170.0,170.0,-15.0,-15.0,-17.0,-25.0,-25.0,-21.0,40.0,50.0
1,2024-08-26 14:34:28,90.0,EXCW（益达西瓜-NCS）,Sugarfree,35.13,35.20,35.20,35.200000,35.200000,0.0,...,170.0,170.0,-15.0,-15.0,-17.0,-25.0,-25.0,-21.0,40.0,50.0
2,2024-08-26 14:42:37,95.0,EXCW（益达西瓜-NCS）,Sugarfree,35.03,35.13,NaN,35.165000,35.165000,0.0,...,182.7,170.0,-20.0,-15.0,-17.0,-25.0,-25.0,-21.0,40.0,50.0
3,2024-08-26 14:52:52,100.0,EXCW（益达西瓜-NCS）,Sugarfree,35.37,35.03,NaN,35.030000,35.120000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-25.0,-20.9,-21.0,51.0,50.0
4,2024-08-26 14:55:36,100.0,EXCW（益达西瓜-NCS）,Sugarfree,35.30,35.37,35.37,35.200000,35.182500,0.0,...,182.7,182.7,-20.0,-20.0,-20.2,-20.3,-20.7,-20.9,51.2,51.0
5,2024-08-26 15:02:20,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.22,35.30,NaN,35.335000,35.206000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.2,-20.8,-20.7,50.9,51.2
6,2024-08-26 15:02:25,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.22,35.22,35.22,35.296667,35.208333,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.3,-20.8,-20.8,50.9,50.9
7,2024-08-26 15:04:00,105.0,EXCW（益达西瓜-NCS）,Sugarfree,35.20,35.22,35.22,35.277500,35.210000,0.0,...,182.7,182.7,-20.0,-20.0,-20.3,-20.3,-20.8,-20.8,50.9,50.9


In [59]:
# 保存 merge_final 到 output_file
# merge_final.to_csv(output_file, index=False)

# Step 6: 获取最新一条数据（离当前时间最近的一条）
latest_data = merge_final.sort_values(by='DataTime', ascending=False).iloc[0]

# 生成 dictionary {列名：当前值}
latest_dict = latest_data.to_dict()

latest_dict

{'DataTime': Timestamp('2024-08-26 15:04:00'),
 'Load': 105.0,
 'Item': 'EXCW（益达西瓜-NCS）',
 'Sugar': 'Sugarfree',
 'Weight': 35.2,
 'Prev_Weight': 35.22,
 'Avg_Weight_5min': 35.22,
 'Avg_Weight_15min': 35.277499999999996,
 'Avg_Weight_30min': 35.21,
 'SFBMix.plcSFBMix.dbAdditionalParameter.StateFromSheeting.bMachineRunning': 0.0,
 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches': 0.069,
 'Prev_CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap3rdSizing.rActualPosition_inches': 0.069,
 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches': 0.075,
 'Prev_CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap2ndSizing.rActualPosition_inches': 0.075,
 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap1stSizing.rActualPosition_inches': 0.109,
 'Prev_CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_Gap1stSizing.rActualPosition_inches': 0.109,
 'CG_Sheeting.CG_Sheeting.dbHMI.Sheeting.SRV_GapFinalSizing.rActualPosition_inches': 0.065,
 'Prev_CG_Sheeting.CG_